In [ ]:
import sys
import os.path
import pylab as plt
plt.style.use('dark_background')
#path = '../'
#sys.path.append(path)
from zedstat.zedstat import zedstat
import pandas as pd

In [ ]:
df=pd.read_csv('examples_data/roc2.csv')
df=df.sort_values('fpr')
#df=df.drop('threshold',axis=1)

In [ ]:
zt=zedstat(df=df,order=3,total_samples=100000,positive_samples=1000,alpha=0.1,prevalence=.01)

In [ ]:
zt.smooth(STEP=0.001)
zt.convexify()
zt.allmeasures(interpolate=True)
zt.correctPPV()
zt.usample(precision=3)
zt.cb_delta()
zt.getUL(direction='L')
zt.getUL(direction='U')

In [ ]:
ax=zt.df_lim['U'].tpr.plot()
ax=zt.df_lim['L'].tpr.plot(ax=ax)
ax=zt.get().tpr.plot(ax=ax)


In [ ]:
wf=zt.get()

In [ ]:
wf.plot(x='LR-',y='LR+')

In [ ]:
zt.operating_zone(LRminus=.65)

In [ ]:
zt.auc_cb2()

In [ ]:
zt.auc()

In [ ]:
rf,txt=zt.interpret(fpr=0.008,number_of_positives=10)
txt

In [ ]:
zt.samplesize(delta_auc=.1)

In [ ]:
import numpy as np
pd.DataFrame(np.array([zt.samplesize(delta_auc=a) 
                       for a in np.arange(.05,.1,.001)]),
             np.arange(.05,.1,.001)).plot(logy=True)

In [ ]:
zt.pvalue()